In [80]:
import numpy as np
import pandas as pd

from cities.utils.data_grabber import DataGrabber
from cities.utils.cleaning_utils import find_repo_root
root = find_repo_root()
data = DataGrabber()


data.get_features_wide(["gdp", "population", "transport",
            "spending_transportation", "spending_commerce", "spending_HHS", "ethnic_composition"])
gdp = data.wide['gdp']

ethnic_composition = data.wide['ethnic_composition']

# population = data.wide['population']
# trnasport = data.wide['transport']
# spending_transportation = data.wide['spending_transportation']
# spending_commerce = data.wide['spending_commerce']

def tableInfo(tableName):

    print(tableName.head())
    print(tableName.dtypes)
    print(f'Number of rows: {tableName.shape[0]}')
    print(f'Unique FIPS numbers {tableName['GeoFIPS'].nunique()}')

In [3]:
metro_areas = pd.read_csv(f"{root}/data/raw/metrolist.csv")

unique_metro_areas = metro_areas["GeoName"].nunique()
print(f"Number of unique metropolitan areas {unique_metro_areas}")

unique_geo_names = metro_areas["CountyName"].nunique()
print(f"Number of unique Counties {unique_geo_names}")
metro_areas.head()

Number of unique metropolitan areas 384
Number of unique Counties 1160


,GeoFIPS,GeoName,CountyFIPS,CountyName
0,10180,"Abilene, TX (MA)",48059,"Callahan, TX"
1,10180,"Abilene, TX (MA)",48253,"Jones, TX"
2,10180,"Abilene, TX (MA)",48441,"Taylor, TX"
3,10420,"Akron, OH (MA)",39133,"Portage, OH"
4,10420,"Akron, OH (MA)",39153,"Summit, OH"


In [26]:
# let's treat as a source of truth for metro areas

geo_names_not_in_gdp = metro_areas[~metro_areas["CountyName"].isin(gdp["GeoName"])]
print(
    f"Number of unique Counties not in GDP: {geo_names_not_in_gdp['CountyName'].nunique()}"
)

geo_fips_not_in_gdp = metro_areas[~metro_areas["CountyFIPS"].isin(gdp["GeoFIPS"])]
print(
    f"Number of unique GeoFIPS not in GDP: {geo_fips_not_in_gdp['CountyFIPS'].nunique()}"
)


geo_fips_5 = geo_names_not_in_gdp[
    ~geo_names_not_in_gdp["CountyFIPS"].isin(geo_fips_not_in_gdp["CountyFIPS"])
]

geo_fips_5

Number of unique Counties not in GDP: 30
Number of unique GeoFIPS not in GDP: 25


,GeoFIPS,GeoName,CountyFIPS,CountyName
153,14500,"Boulder, CO (MA)",8013,"Boulder, CO"
327,19740,"Denver-Aurora-Lakewood, CO (MA)",8001,"Adams, CO"
335,19740,"Denver-Aurora-Lakewood, CO (MA)",8059,"Jefferson, CO"
427,24540,"Greeley, CO (MA)",8123,"Weld, CO"
642,31340,"Lynchburg, VA (MA)",51019,"Bedford, VA"


In [28]:
values_to_check = geo_fips_5["CountyFIPS"].values
rows_matching_values = gdp[gdp["GeoFIPS"].isin(values_to_check)]

rows_matching_values

# TODO_NIKDOEM add * to the counties above

,GeoFIPS,GeoName,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2013,2014,2015,2016,2017,2018,2019,2020,2021
237,8001,"Adams, CO*",76.029,77.712,80.399,83.863,86.430,88.808,91.488,102.632,...,99.005,103.905,112.515,116.871,121.798,126.565,131.633,137.401,137.953,146.579
243,8013,"Boulder, CO*",86.496,80.695,81.918,84.258,86.107,88.488,94.111,98.539,...,99.609,103.041,106.133,110.350,113.629,120.183,123.812,132.409,129.312,139.504
266,8059,"Jefferson, CO*",98.320,97.876,95.805,94.062,96.785,95.389,97.371,100.633,...,99.513,101.220,103.917,107.623,112.683,115.371,119.537,125.041,121.914,128.158
298,8123,"Weld, CO*",61.847,61.231,63.658,68.305,73.787,75.629,78.809,83.712,...,90.961,112.153,135.644,153.223,160.641,172.257,188.257,198.269,185.268,172.810
2809,51019,"Bedford, VA*",82.829,83.962,89.261,89.915,92.256,96.818,97.012,99.789,...,99.939,102.505,101.023,103.002,100.482,100.840,102.389,101.057,98.323,100.298


In [16]:
cbs_list = pd.read_csv(f"{root}/data/raw/cbsalist.csv")


msa_table = cbs_list[
    cbs_list["metropolitanmicropolitanstatis"] == "Metropolitan Statistical Area"
]
non_msa_table = cbs_list[
    cbs_list["metropolitanmicropolitanstatis"] == "Micropolitan Statistical Area"
]

display(msa_table.head(2))
display(non_msa_table.head(2))

assert msa_table.shape[0] + non_msa_table.shape[0] == cbs_list.shape[0]

print(msa_table.shape[0])

# cbsatitle is a name of the metropolitan area

,cbsacode,metropolitandivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,metropolitandivisiontitle,csatitle,countycountyequivalent,statename,fipsstatecode,fipscountycode,centraloutlyingcounty
0,33860,NaN,388.0,"Montgomery, AL",Metropolitan Statistical Area,NaN,"Montgomery-Selma, AL",Autauga County,Alabama,1,1,Central
1,19300,NaN,380.0,"Daphne-Fairhope-Foley, AL",Metropolitan Statistical Area,NaN,"Mobile-Daphne-Fairhope, AL",Baldwin County,Alabama,1,3,Central


,cbsacode,metropolitandivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,metropolitandivisiontitle,csatitle,countycountyequivalent,statename,fipsstatecode,fipscountycode,centraloutlyingcounty
2,21640,NaN,NaN,"Eufaula, AL-GA",Micropolitan Statistical Area,NaN,NaN,Barbour County,Alabama,1,5,Central
6,29300,NaN,122.0,"LaGrange, GA-AL",Micropolitan Statistical Area,NaN,"Atlanta--Athens-Clarke County--Sandy Springs, ...",Chambers County,Alabama,1,17,Outlying


1252
